# Filter Variants

This notebook is meant to filter the rat variants data even further following the LD pruning step.  The filtering will be similar to the way it was conducted in Xu et al., Nature

In [1]:
import sgkit as sg
from sgkit.io import plink
import pandas as pd
import numpy as np
import math
import pickle
import xarray as xr

## Load in Genotypes and Phenotypes Data

In [9]:
ds = plink.read_plink(bed_path = 'ratgenes_pruned/ratgenes_pruned.bed', bim_path = 'ratgenes_pruned/ratgenes_pruned.bim', fam_path = 'ratgenes_pruned/ratgenes_pruned.fam')
call_g_mask = ds["call_genotype_mask"].any(dim = "ploidy")
call_g = xr.where(call_g_mask, -1, ds["call_genotype"].sum(dim = "ploidy"))
genotypes_matrix = call_g.values
genotypes_matrix = np.transpose(genotypes_matrix)

loco = pd.read_csv("pheno_loco_clean.txt", sep = '\t')
ds

<xarray.Dataset>
Dimensions:             (variants: 96654, alleles: 2, samples: 13526, ploidy: 2)
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_contig      (variants) int16 0 0 0 0 0 0 0 ... 20 20 20 20 20 20 20
    variant_position    (variants) int32 dask.array<chunksize=(96654,), meta=np.ndarray>
    variant_allele      (variants, alleles) |S1 dask.array<chunksize=(96654, 1), meta=np.ndarray>
    sample_id           (samples) <U15 dask.array<chunksize=(13526,), meta=np.ndarray>
    call_genotype       (variants, samples, ploidy) int8 dask.array<chunksize=(8192, 8192, 2), meta=np.ndarray>
    call_genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(8192, 8192, 2), meta=np.ndarray>
    variant_id          (variants) <U15 dask.array<chunksize=(96654,), meta=np.ndarray>
    sample_family_id    (samples) <U15 dask.array<chunksize=(13526,), meta=np.ndarray>
    sample_paternal_id  (samples) <U4 dask.array<chunksize=(13526,), meta=np.ndarray>
    sample_maternal_id  (samples) <U4 dask.array<chunksize=(13526,), meta=np.ndarray>
    sample_sex          (samples) int8 dask.array<chunksize=(13526,), meta=np.ndarray>
    sample_phenotype    (samples) int8 dask.array<chunksize=(13526,), meta=np.ndarray>
Attributes:
    contigs:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',...
    source:   sgkit-0.6.0

## Clean phenotypes data

In [10]:
def get_rats_with_loco(ds, phenotypes):
    
    rat_ids = ds['sample_id'].values
    phenotypes = phenotypes[(phenotypes['rfid'].isin(rat_ids))]
    phenotypes = phenotypes.set_index('rfid')
    phenotypes.index.name = 'Sample'
    select_traits = ['loco_maxcent', 'loco_maxdis', 'loco_maxrear', 'loco_maxact']
    phenotypes = phenotypes[select_traits]
    
    return(phenotypes)

In [11]:
loco = get_rats_with_loco(ds, loco)
print(loco.shape)
loco.head()

(2368, 4)


,loco_maxcent,loco_maxdis,loco_maxrear,loco_maxact
Sample,,,,
00077E61F3,171.3,568.0,19.0,109.0
00077E6207,115.4,697.0,26.0,135.0
00077E6232,48.1,652.0,20.0,124.0
00077E6239,147.1,466.0,25.0,97.0
00077E62D2,89.4,599.0,22.0,96.0


## Merge Genotypes with Phenotypes

In [30]:
ds_annotations = pd.DataFrame.to_xarray(loco).rename({"Sample":"samples"})
ds = ds.set_index({"samples": "sample_id"})
ds = ds.drop_duplicates(dim = ['samples'])
ds = ds.sel(samples = list(loco.index))
ds = ds.merge(ds_annotations, join="left")
ds

/Users/zacharywallace/anaconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing with an out-of-order index is generating 74 times more chunks
  return self.array[key]


<xarray.Dataset>
Dimensions:             (variants: 96654, alleles: 2, samples: 2368, ploidy: 2)
Coordinates:
  * samples             (samples) object '00077E61F3' ... 'E11DCD33DD'
Dimensions without coordinates: variants, alleles, ploidy
Data variables: (12/15)
    variant_contig      (variants) int16 0 0 0 0 0 0 0 ... 20 20 20 20 20 20 20
    variant_position    (variants) int32 dask.array<chunksize=(96654,), meta=np.ndarray>
    variant_allele      (variants, alleles) |S1 dask.array<chunksize=(96654, 1), meta=np.ndarray>
    call_genotype       (variants, samples, ploidy) int8 dask.array<chunksize=(8192, 38, 2), meta=np.ndarray>
    call_genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(8192, 38, 2), meta=np.ndarray>
    variant_id          (variants) <U15 dask.array<chunksize=(96654,), meta=np.ndarray>
    ...                  ...
    sample_sex          (samples) int8 dask.array<chunksize=(2368,), meta=np.ndarray>
    sample_phenotype    (samples) int8 dask.array<chunksize=(2368,), meta=np.ndarray>
    loco_maxcent        (samples) float64 171.3 115.4 48.1 ... 36.7 117.6 180.0
    loco_maxdis         (samples) float64 568.0 697.0 652.0 ... 534.0 550.0
    loco_maxrear        (samples) float64 19.0 26.0 20.0 25.0 ... 37.0 31.0 28.0
    loco_maxact         (samples) float64 109.0 135.0 124.0 ... 77.0 83.0 95.0
Attributes:
    contigs:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',...
    source:   sgkit-0.6.0

## Begin Filterig the Genotypes Dataset

### First fillter out missing SNPs for GWAS

In [32]:
ds = sg.stats.pca.count_call_alternate_alleles(ds)
variant_mask = ((ds.call_alternate_allele_count < 0).any(dim="samples")) | \
    (ds.call_alternate_allele_count.std(dim="samples") <= 0.0)
ds = ds.sel(variants=~variant_mask)
print(f"Samples: {len(ds.samples)}  Variants: {len(ds.variants)}")
ds

Samples: 2368  Variants: 8690


<xarray.Dataset>
Dimensions:                      (samples: 2368, variants: 8690, alleles: 2,
                                  ploidy: 2)
Coordinates:
  * samples                      (samples) object '00077E61F3' ... 'E11DCD33DD'
Dimensions without coordinates: variants, alleles, ploidy
Data variables: (12/16)
    call_alternate_allele_count  (variants, samples) int16 dask.array<chunksize=(625, 38), meta=np.ndarray>
    variant_contig               (variants) int16 0 0 0 0 0 0 ... 20 20 20 20 20
    variant_position             (variants) int32 dask.array<chunksize=(8690,), meta=np.ndarray>
    variant_allele               (variants, alleles) |S1 dask.array<chunksize=(8690, 1), meta=np.ndarray>
    call_genotype                (variants, samples, ploidy) int8 dask.array<chunksize=(625, 38, 2), meta=np.ndarray>
    call_genotype_mask           (variants, samples, ploidy) bool dask.array<chunksize=(625, 38, 2), meta=np.ndarray>
    ...                           ...
    sample_sex                   (samples) int8 dask.array<chunksize=(2368,), meta=np.ndarray>
    sample_phenotype             (samples) int8 dask.array<chunksize=(2368,), meta=np.ndarray>
    loco_maxcent                 (samples) float64 171.3 115.4 ... 117.6 180.0
    loco_maxdis                  (samples) float64 568.0 697.0 ... 534.0 550.0
    loco_maxrear                 (samples) float64 19.0 26.0 20.0 ... 31.0 28.0
    loco_maxact                  (samples) float64 109.0 135.0 ... 83.0 95.0
Attributes:
    contigs:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',...
    source:   sgkit-0.6.0

In [34]:
# Save filtered to pickle

with open('ds_known.pkl', 'wb') as handle:
    pickle.dump(ds, handle)

In [2]:
# Open saved pickled file

with open('ds_known.pkl', 'rb') as handle:
    ds = pickle.load(handle)

### Run GWAS Linear Regression to get Pvalues

In [4]:
traits = ['loco_maxcent', 'loco_maxdis', 'loco_maxrear', 'loco_maxact']
ds["call_dosage"] = ds.call_genotype.sum(dim="ploidy")
ds_lr = sg.gwas_linear_regression(ds, dosage="call_dosage", add_intercept=True, covariates=[], traits=traits)
ds_lr

<xarray.Dataset>
Dimensions:                      (variants: 8690, traits: 4, samples: 2368,
                                  alleles: 2, ploidy: 2)
Coordinates:
  * samples                      (samples) object '00077E61F3' ... 'E11DCD33DD'
Dimensions without coordinates: variants, traits, alleles, ploidy
Data variables: (12/20)
    variant_linreg_beta          (variants, traits) float64 dask.array<chunksize=(625, 4), meta=np.ndarray>
    variant_linreg_t_value       (variants, traits) float64 dask.array<chunksize=(625, 4), meta=np.ndarray>
    variant_linreg_p_value       (variants, traits) float64 dask.array<chunksize=(625, 4), meta=np.ndarray>
    call_alternate_allele_count  (variants, samples) int16 dask.array<chunksize=(625, 38), meta=np.ndarray>
    variant_contig               (variants) int16 0 0 0 0 0 0 ... 20 20 20 20 20
    variant_position             (variants) int32 dask.array<chunksize=(8690,), meta=np.ndarray>
    ...                           ...
    sample_phenotype             (samples) int8 dask.array<chunksize=(2368,), meta=np.ndarray>
    loco_maxcent                 (samples) float64 171.3 115.4 ... 117.6 180.0
    loco_maxdis                  (samples) float64 568.0 697.0 ... 534.0 550.0
    loco_maxrear                 (samples) float64 19.0 26.0 20.0 ... 31.0 28.0
    loco_maxact                  (samples) float64 109.0 135.0 ... 83.0 95.0
    call_dosage                  (variants, samples) int64 dask.array<chunksize=(625, 38), meta=np.ndarray>
Attributes:
    contigs:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',...
    source:   sgkit-0.6.0

### Check some of the pvalue stats, Make sure no NaNs

In [36]:
pvals = np.asarray(ds_lr.variant_linreg_p_value)
print("Length of set", len(pvals))
print("Some pvalues", pvals[0:10])

Length of set 8690
Some pvalues [[0.83498755 0.83178252 0.39806148 0.78515391]
 [0.83498755 0.83178252 0.39806148 0.78515391]
 [0.83498755 0.83178252 0.39806148 0.78515391]
 [0.48890551 0.18730565 0.2468513  0.30857111]
 [0.48890551 0.18730565 0.2468513  0.30857111]
 [0.6104458  0.04121454 0.04754996 0.00912043]
 [0.24086775 0.22143437 0.04632651 0.32791972]
 [0.61826744 0.11577944 0.07347522 0.01856175]
 [0.61826744 0.11577944 0.07347522 0.01856175]
 [0.11371496 0.17127835 0.89635393 0.26219276]]


### Filter based on 0.05 GWAS Pvalues

In [37]:
ds_lr_05 = ds_lr.sel(variants=((ds_lr.variant_linreg_p_value < 0.05).any('traits')))
print(f"Samples: {len(ds_lr_05.samples)}  Variants: {len(ds_lr_05.variants)}")

Samples: 2368  Variants: 3408


### Filter based on 0.01 GWAS Pvalues

In [38]:
ds_lr_01 = ds_lr_05.sel(variants=((ds_lr_05.variant_linreg_p_value < 0.01).any('traits')))
print(f"Samples: {len(ds_lr_01.samples)}  Variants: {len(ds_lr_01.variants)}")

Samples: 2368  Variants: 1319


### Filter based on 0.001 GWAS Pvalues

In [39]:
ds_lr_001 = ds_lr_01.sel(variants=((ds_lr_01.variant_linreg_p_value < 0.001).any('traits')))
print(f"Samples: {len(ds_lr_001.samples)}  Variants: {len(ds_lr_001.variants)}")

Samples: 2368  Variants: 546


### Filter based on 0.0001 GWAS Pvalues

In [40]:
ds_lr_0001 = ds_lr_001.sel(variants=((ds_lr_001.variant_linreg_p_value < 0.0001).any('traits')))
print(f"Samples: {len(ds_lr_0001.samples)}  Variants: {len(ds_lr_0001.variants)}")

Samples: 2368  Variants: 152


### Filter based on 0.00001 GWAS Pvalues

In [41]:
ds_lr_00001 = ds_lr_0001.sel(variants=((ds_lr_0001.variant_linreg_p_value < 0.00001).any('traits')))
print(f"Samples: {len(ds_lr_00001.samples)}  Variants: {len(ds_lr_00001.variants)}")

Samples: 2368  Variants: 100


## Save as Numpy Arrays and pickle

In [45]:
call_g_mask = ds_lr_05["call_genotype_mask"].any(dim = "ploidy")
call_g = xr.where(call_g_mask, -1, ds_lr_05["call_genotype"].sum(dim = "ploidy"))
genotypes_matrix_05 = call_g.values
genotypes_matrix_05 = np.transpose(genotypes_matrix_05)

with open('genotypes_matrix_05.pkl', 'wb') as handle:
    pickle.dump(genotypes_matrix_05, handle)